In [37]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd

In [39]:
data = pd.read_csv('wpbc.data', header=None)

In [ ]:
y = (data[1] == "N").astype(np.float64)
X = np.array(data[[2,3,4,5,6]], np.float64)
XX = X/np.mean(X,0)
Sigma_beta = np.eye(5).astype(np.float64)

In [278]:
def U(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] @ theta * y[i] - np.log(1+np.exp(X[i] @ theta)))
    return -a + 0.5 * theta.T @ np.linalg.inv(Sigma_beta) @ theta

In [616]:
def gradU(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] * y[i] - (X[i] * np.exp(X[i] @ theta)) / (1 + np.exp(X[i] @ theta)))
    return -a + np.linalg.inv(Sigma_beta) @ theta

In [623]:
bb = np.array([0,0,0,0,0]).astype(np.float64)

In [625]:
def hmc_vector(U, gradU, m, dt, nstep, x, X, y, Sigma_beta):
    mean = np.zeros_like(x)
    p = np.random.multivariate_normal(mean, m)
    oldX = x.copy()
    oldEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    for i in range(nstep):
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
        x += np.linalg.solve(m, p) * dt
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
    newEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    if np.random.random() > np.exp(oldEnergy - newEnergy): # Metropolis-Hastings
        x = oldX.copy()
    return x.copy()

In [636]:
nsample = 8000
m = np.eye(5)
dt = 0.1
nstep = 10

In [ ]:
samples = []
bb = np.zeros_like(bb)
for i in range(nsample):
    bb = hmc_vector(U, gradU, m, dt, nstep, bb, XX, y, Sigma_beta)
    print(i)
    samples.append(bb)

In [640]:
samples = np.array(samples)
samples.mean(axis=0)

array([ 1.35263749, -0.2070946 ,  0.96616421, -0.2160136 , -0.41400247])

In [2]:
%load_ext rpy2.ipython

In [ ]:
%%R -i XX,y

library(R2jags)
dat = read.csv('wpbc.csv',header=F)


jags_data = list(Y = y,
                 X = XX,
                 n = length(y),
                 p = ncol(XX))

jags.model = "model {
  
  for (i in 1:n) {
    logit(mu[i]) = inprod(X[i,], theta)
    Y[i] ~ dbern(mu[i])
  }
  
  for (j in 1:(p)){
    theta[j] ~ dnorm(0,1)
  }
}
"

parameters = "theta"

jags_sim = jags(jags_data, inits=NULL, 
                parameters.to.save=parameters,
                model.file=textConnection(jags.model),
                n.iter=10000)

jags_sim

```
> jags_sim
Inference for Bugs model at "9", fit using jags,
 3 chains, each with 10000 iterations (first 5000 discarded), n.thin = 5
 n.sims = 3000 iterations saved
         mu.vect sd.vect    2.5%     25%     50%     75%   97.5%  Rhat
theta[1]   1.347   0.289   0.808   1.155   1.340   1.532   1.959 1.002
theta[2]  -0.241   0.829  -1.868  -0.804  -0.240   0.299   1.382 1.001
theta[3]   0.973   0.551  -0.089   0.588   0.962   1.324   2.091 1.002
theta[4]  -0.220   0.841  -1.831  -0.777  -0.237   0.345   1.436 1.003
theta[5]  -0.394   0.629  -1.662  -0.810  -0.399   0.036   0.831 1.002
deviance 186.465   2.588 183.346 184.731 185.909 187.563 192.821 1.001
         n.eff
theta[1]  1800
theta[2]  3000
theta[3]  1500
theta[4]   800
theta[5]  1900
deviance  3000
```